In [1]:
# call relevant packages
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *

import sparknlp

# to use GPU 
spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  2.7.3
Apache Spark version:  2.4.4


### Creating a composite CoNLL file 

In [2]:
# create the training file
with open("/Users/ramybal/Downloads/bio/train/onto.bn.ner") as fp:
    text = fp.readlines()

In [3]:
text = "".join(text[1:]).split("\n\n") 

In [4]:
text[:10]

['This\tDT\t(TOP(S(NP*)\tO\nis\tVBZ\t(VP*\tO\nThe\tDT\t(NP(NP*\tB-ORG\nWorld\tNNP\t*)\tI-ORG\n,\t,\t*\tO\na\tDT\t(NP(NP*\tO\nco-production\tNN\t*)\tO\nof\tIN\t(PP*\tO\nthe\tDT\t(NP(NP*\tB-ORG\nBBC\tNNP\t*\tI-ORG\nWorld\tNNP\t*\tI-ORG\nService\tNNP\t*)\tI-ORG\n,\t,\t*\tO\nPRI\tNNP\t(NP*)\tB-ORG\n,\t,\t*\tO\nand\tCC\t*\tO\nWGBH\tNNP\t(NP(NP*)\tB-ORG\nin\tIN\t(PP*\tO\nBoston\tNNP\t(NP*))))))))\tB-GPE\n.\t.\t*))\tO',
 'I\tPRP\t(TOP(S(NP*)\tO\nam\tVBP\t(VP*\tO\nLisa\tNNP\t(NP*\tB-PERSON\nMullins\tNNP\t*))\tI-PERSON\n.\t.\t*))\tO',
 "A\tDT\t(TOP(S(NP*\tO\nplan\tNN\t*)\tO\nwas\tVBD\t(VP*\tO\nannounced\tVBN\t(VP*\tO\ntoday\tNN\t(NP*)\tB-DATE\nto\tTO\t(S(VP*\tO\nraise\tVB\t(VP*\tO\nthe\tDT\t(NP(NP*\tO\nRussian\tJJ\t*\tB-NORP\nnuclear\tJJ\t*\tO\nsubmarine\tNN\t*)\tO\n`\t''\t(NP*\tO\nKursk\tNNP\t*\tB-PRODUCT\n'\t''\t*))\tO\n,\t,\t*\tO\nnext\tJJ\t(NP(NP*\tB-DATE\nsummer\tNN\t*)\tI-DATE\n,\t,\t*\tO\nalmost\tRB\t(SBAR(NP(QP*\tB-DATE\na\tDT\t*)\tI-DATE\nyear\tNN\t*)\tI-DATE\nafter\tIN\t*\tO\nit\tPRP\

In [5]:
import pandas as pd
df = pd.DataFrame([x.split('\t') for x in text[1].split('\n')], columns=["Token","Pos","Pos_special","Entity_label"])

In [6]:
df

,Token,Pos,Pos_special,Entity_label
0,I,PRP,(TOP(S(NP*),O
1,am,VBP,(VP*,O
2,Lisa,NNP,(NP*,B-PERSON
3,Mullins,NNP,*)),I-PERSON
4,.,.,*)),O


In [7]:
len(text)

10487

In [8]:
# creating the training data
conll_lines = "-DOCSTART- -X- -X- -O-\n\n"
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [9]:
# adding other news files
with open("/Users/ramybal/Downloads/bio/train/onto.mz.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [10]:
len(text)

7321

In [11]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [12]:
# adding other news files
with open("/Users/ramybal/Downloads/bio/train/onto.nw.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [13]:
len(text)

16034

In [14]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [15]:
# adding broadcasting file
with open("/Users/ramybal/Downloads/bio/train/onto.bc.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [16]:
len(text)

10713

In [17]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [18]:
# adding telephonic conversation files
with open("/Users/ramybal/Downloads/bio/train/onto.tc.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [19]:
len(text)

11273

In [20]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [21]:
# adding web log file
with open("/Users/ramybal/Downloads/bio/train/onto.wb.ner") as fp:
    text = fp.readlines()
text = "".join(text[1:]).split("\n\n")

In [22]:
len(text)

6585

In [23]:
for t in range(len(text)):    
    df = pd.DataFrame([x.split('\t') for x in text[t].split('\n') if len(x.split('\t')) == 4], columns=["Token","Pos","Pos_special","Entity_label"])
    tokens = df.Token.tolist()
    pos_labels = df.Pos.tolist()
    entity_labels = df.Entity_label.tolist()
    for token, pos, label in zip(tokens,pos_labels,entity_labels):
        conll_lines += "{} {} {} {}\n".format(token, pos, pos, label)
    conll_lines += "\n"

In [24]:
with open("/Users/ramybal/Downloads/bio/train/sample.train","w") as fp:
    for line in conll_lines:
        fp.write(line)

### import data in CONLL format

In [2]:
from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, '/Users/ramybal/Downloads/bio/train/sample.train')
training_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|This is The World...|[[document, 0, 88...|[[document, 0, 88...|[[token, 0, 3, Th...|[[pos, 0, 3, DT, ...|[[named_entity, 0...|
| I am Lisa Mullins .|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 0, I,...|[[pos, 0, 0, PRP,...|[[named_entity, 0...|
|A plan was announ...|[[document, 0, 18...|[[document, 0, 18...|[[token, 0, 0, A,...|[[pos, 0, 0, DT, ...|[[named_entity, 0...|
|The Kursk Foundat...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 2, Th...|[[pos, 0, 2, DT, ...|[[named_entity, 0...|
|The BBC 's James ...|[[document, 0, 66...|[[document, 0, 66...|[[token, 0, 2, Th...|[[pos, 0, 2, DT, ..

### Training the model - 2 epochs

In [3]:
# initialize embeddings
bert = BertEmbeddings.pretrained('bert_base_cased', 'en').setInputCols(["sentence",'token']).setOutputCol("bert").setCaseSensitive(False)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [64]:
# transforming the training data into embeddings and saving it as parquet files
readyTrainingData = bert.transform(training_data)

readyTrainingData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_train")

In [65]:
readyTrainingData = spark.read.parquet("/tmp/conll2003/bert_train")

In [66]:
# initialize NER tagger
nerTagger = NerDLApproach()\
.setInputCols(["sentence", "token", "bert"])\
.setLabelColumn("label")\
.setOutputCol("ner")\
.setMaxEpochs(2)\
.setBatchSize(2)\
.setEnableMemoryOptimizer(True)\
.setRandomSeed(0)\
.setVerbose(1)\
.setValidationSplit(0.2)\
.setEvaluationLogExtended(True)\
.setEnableOutputLogs(True)\
.setIncludeConfidence(True)\
.setTestDataset("test_withEmbeds.parquet")

In [67]:
# first use-case with test data as news
test = CoNLL().readDataset(spark, "/Users/ramybal/Downloads/bio/test/sample.train")

In [68]:
test = bert.transform(test)

In [70]:
test.write.parquet("test_withEmbeds.parquet")

In [71]:
# train the model
%time myNerModel = nerTagger.fit(readyTrainingData)

CPU times: user 335 ms, sys: 250 ms, total: 585 ms
Wall time: 1h 14min 10s


In [72]:
# infer from the trained model
%time results = myNerModel.transform(test).select("sentence","token","label","ner").collect()

CPU times: user 2.41 s, sys: 1.93 s, total: 4.35 s
Wall time: 4min 57s


In [73]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [74]:
count

2

In [75]:
indices

[3584, 3772]

In [76]:
exclusion_list = [results[3584],results[3772]]

In [77]:
results = [results[i] for i in range(len(results)) if i not in [3584, 3772]]

In [78]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [79]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9577009353150293


In [80]:
print(f1_score(labels,ners))

0.7584643848288621


In [82]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.79      0.81      0.80       935
        DATE       0.81      0.75      0.78      1600
       EVENT       0.50      0.25      0.34        63
         FAC       0.69      0.18      0.28       135
         GPE       0.80      0.86      0.83      2240
    LANGUAGE       1.00      0.23      0.37        22
         LAW       1.00      0.00      0.00        40
         LOC       0.63      0.30      0.40       179
       MONEY       0.87      0.84      0.86       314
        NORP       0.79      0.79      0.79       841
     ORDINAL       0.71      0.90      0.79       195
         ORG       0.76      0.54      0.63      1791
     PERCENT       0.85      0.86      0.86       346
      PERSON       0.84      0.83      0.83      1988
     PRODUCT       0.60      0.37      0.46        76
    QUANTITY       0.55      0.57      0.56       105
        TIME       0.53      0.45      0.49       212
 WORK_OF_ART       0.45    

### Training the model - 3 epochs

In [4]:
# transforming the training data into embeddings and saving it as parquet files
readyTrainingData = bert.transform(training_data)

readyTrainingData.write.mode("Overwrite").parquet("/tmp/conll2003/bert_train")

In [5]:
readyTrainingData = spark.read.parquet("/tmp/conll2003/bert_train")

In [6]:
# initialize NER tagger
nerTagger = NerDLApproach()\
.setInputCols(["sentence", "token", "bert"])\
.setLabelColumn("label")\
.setOutputCol("ner")\
.setMaxEpochs(3)\
.setBatchSize(2)\
.setEnableMemoryOptimizer(True)\
.setRandomSeed(0)\
.setVerbose(1)\
.setValidationSplit(0.2)\
.setEvaluationLogExtended(True)\
.setEnableOutputLogs(True)\
.setIncludeConfidence(True)\
.setTestDataset("test_withEmbeds.parquet")

In [7]:
# first use-case with test data as news
test = CoNLL().readDataset(spark, "/Users/ramybal/Downloads/bio/test/sample.train")

In [8]:
test = bert.transform(test)

In [10]:
test.write.parquet("test_withEmbeds.parquet")

In [11]:
# train the model
%time myNerModel = nerTagger.fit(readyTrainingData)

CPU times: user 503 ms, sys: 371 ms, total: 874 ms
Wall time: 1h 57min 38s


In [12]:
# infer from the trained model
%time results = myNerModel.transform(test).select("sentence","token","label","ner").collect()

CPU times: user 2.79 s, sys: 471 ms, total: 3.26 s
Wall time: 5min 40s


In [13]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [14]:
print(count)
print(indices)

2
[3584, 3772]


In [15]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [16]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [17]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9602327137253873


In [18]:
print(f1_score(labels,ners))

0.7764214729772657


In [19]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.80      0.83      0.82       935
        DATE       0.79      0.80      0.80      1600
       EVENT       0.59      0.27      0.37        63
         FAC       0.67      0.25      0.37       135
         GPE       0.83      0.88      0.85      2240
    LANGUAGE       1.00      0.18      0.31        22
         LAW       1.00      0.00      0.00        40
         LOC       0.66      0.41      0.50       179
       MONEY       0.84      0.82      0.83       314
        NORP       0.78      0.82      0.80       841
     ORDINAL       0.73      0.89      0.80       195
         ORG       0.73      0.62      0.67      1791
     PERCENT       0.84      0.86      0.85       346
      PERSON       0.85      0.82      0.84      1988
     PRODUCT       0.65      0.32      0.42        76
    QUANTITY       0.65      0.67      0.66       105
        TIME       0.56      0.50      0.53       212
 WORK_OF_ART       0.35    

### Use pre-trained model

In [29]:
ner_onto = NerDLModel.pretrained('onto_bert_base_cased', lang='en') \
        .setInputCols(["sentence", "token", "bert"])\
        .setOutputCol("ner")

onto_bert_base_cased download started this may take some time.
Approximate size to download 15.5 MB
[OK!]


In [70]:
# transform the model
myNerModel = ner_onto.transform(readyTrainingData)

In [88]:
# tabulate the results
from sklearn.metrics import classification_report
import pyspark.sql.functions as F

results = myNerModel.select("sentence","token","label","ner").collect()

In [89]:
len(results)

8262

In [90]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [91]:
count

2

In [92]:
indices

[1519, 1707]

In [93]:
print(len(results[1519]['label']))
print(len(results[1519]['ner']))

151
115


In [94]:
print(len(results[1707]['label']))
print(len(results[1707]['ner']))

115
107


In [95]:
exclusion_list = [results[1519],results[1707]]

In [96]:
results = [results[i] for i in range(len(results)) if i not in [1519,1707]]

In [97]:
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [98]:
count

0

In [99]:
for t in results[1]['token']:
    print(t['result'])

My
brother
Qais
,
the
Son
of
his
Father
,
I
thank
you
for
your
post
and
the
addition
,
may
God
take
care
of
you
.


In [100]:
for t in results[1]['label']:
    print(t['result'])

O
O
B-PERSON
I-PERSON
I-PERSON
I-PERSON
I-PERSON
I-PERSON
I-PERSON
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O


In [101]:
for t in results[1]['ner']:
    print(t['result'])

O
O
B-PERSON
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O
O


In [102]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])    

In [103]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9132833099395259


In [104]:
print(f1_score(labels,ners))

0.48666707161513695


In [105]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.79      0.88      0.83       935
        DATE       0.83      0.68      0.75      1600
       EVENT       0.50      0.05      0.09        63
         FAC       0.17      0.01      0.01       135
         GPE       0.81      0.14      0.24      2240
    LANGUAGE       1.00      0.00      0.00        22
         LAW       1.00      0.03      0.05        40
         LOC       0.48      0.08      0.13       179
       MONEY       0.85      0.86      0.85       314
        NORP       0.63      0.23      0.34       841
     ORDINAL       0.65      0.90      0.76       195
         ORG       0.53      0.12      0.19      1791
     PERCENT       0.90      0.88      0.89       346
      PERSON       0.85      0.19      0.31      1988
     PRODUCT       0.57      0.05      0.10        76
    QUANTITY       0.77      0.82      0.80       105
        TIME       0.59      0.61      0.60       212
 WORK_OF_ART       0.77    

### Use pre-trained model with pipeline (bert and NER model)

In [107]:
nlp_pipeline = Pipeline(stages=[bert,ner_onto])

In [108]:
%time myNerModel = nlp_pipeline.fit(training_data)

CPU times: user 166 µs, sys: 198 µs, total: 364 µs
Wall time: 359 µs


In [41]:
%time results = myNerModel.transform(training_data).select("sentence","token","label","ner").collect()

CPU times: user 2.5 s, sys: 235 ms, total: 2.74 s
Wall time: 4min 34s


In [42]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [43]:
count

2

In [44]:
indices

[3584, 3772]

In [45]:
len(results)

8262

In [46]:
exclusion_list = [results[3584],results[3772]]

In [47]:
results = [results[i] for i in range(len(results)) if i not in [3584,3772]]

In [48]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [49]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9132833099395259


In [65]:
print(f1_score(labels,ners))

0.48666707161513695


In [66]:
import numpy as np
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.79      0.88      0.83       935
        DATE       0.83      0.68      0.75      1600
       EVENT       0.50      0.05      0.09        63
         FAC       0.17      0.01      0.01       135
         GPE       0.81      0.14      0.24      2240
    LANGUAGE       1.00      0.00      0.00        22
         LAW       1.00      0.03      0.05        40
         LOC       0.48      0.08      0.13       179
       MONEY       0.85      0.86      0.85       314
        NORP       0.63      0.23      0.34       841
     ORDINAL       0.65      0.90      0.76       195
         ORG       0.53      0.12      0.19      1791
     PERCENT       0.90      0.88      0.89       346
      PERSON       0.85      0.19      0.31      1988
     PRODUCT       0.57      0.05      0.10        76
    QUANTITY       0.77      0.82      0.80       105
        TIME       0.59      0.61      0.60       212
 WORK_OF_ART       0.77    

In [59]:
set([l for label in labels for l in label]) - set([n for ner in ners for n in ner])

{'B-LANGUAGE', 'I-ORDINAL'}

### Use pre-trained Small BERT model

In [20]:
ner_onto = NerDLModel.pretrained('onto_small_bert_L4_512', lang='en') \
        .setInputCols(["sentence", "token", "bert"])\
        .setOutputCol("ner")

onto_small_bert_L4_512 download started this may take some time.
Approximate size to download 14.8 MB
[OK!]


In [22]:
# use small bert embeddings
bert = BertEmbeddings.pretrained('small_bert_L4_512', 'en').setInputCols(["sentence",'token']).setOutputCol("bert").setCaseSensitive(False)

small_bert_L4_512 download started this may take some time.
Approximate size to download 104 MB
[OK!]


In [23]:
nlp_pipeline = Pipeline(stages=[bert,ner_onto])

In [24]:
%time myNerModel = nlp_pipeline.fit(training_data)

CPU times: user 156 µs, sys: 81 µs, total: 237 µs
Wall time: 211 µs


In [25]:
%time results = myNerModel.transform(test).select("sentence","token","label","ner").collect()

CPU times: user 3.54 s, sys: 1.2 s, total: 4.74 s
Wall time: 1min 31s


In [26]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [27]:
print(count)
print(indices)

2
[3584, 3772]


In [28]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [29]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [30]:
from seqeval.metrics import accuracy_score, f1_score, classification_report
print(accuracy_score(labels,ners))

0.9709960514751217


In [31]:
print(f1_score(labels,ners))

0.8455879074330048


In [32]:
import numpy as np
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.84      0.86      0.85       935
        DATE       0.81      0.87      0.84      1600
       EVENT       0.45      0.33      0.38        63
         FAC       0.65      0.57      0.61       135
         GPE       0.94      0.92      0.93      2240
    LANGUAGE       0.91      0.45      0.61        22
         LAW       0.65      0.28      0.39        40
         LOC       0.68      0.64      0.66       179
       MONEY       0.85      0.86      0.86       314
        NORP       0.89      0.91      0.90       841
     ORDINAL       0.77      0.89      0.83       195
         ORG       0.81      0.76      0.79      1791
     PERCENT       0.90      0.90      0.90       346
      PERSON       0.89      0.91      0.90      1988
     PRODUCT       0.62      0.42      0.50        76
    QUANTITY       0.76      0.76      0.76       105
        TIME       0.56      0.56      0.56       212
 WORK_OF_ART       0.56    

### Use pre-trained Electra model

In [33]:
ner_onto = NerDLModel.pretrained('onto_electra_large_uncased', lang='en') \
        .setInputCols(["sentence", "token", "bert"])\
        .setOutputCol("ner")

onto_electra_large_uncased download started this may take some time.
Approximate size to download 16.2 MB
[OK!]


In [34]:
# use elctra embeddings
electra = BertEmbeddings.pretrained('electra_large_uncased', 'en').setInputCols(["sentence",'token']).setOutputCol("bert").setCaseSensitive(False)

electra_large_uncased download started this may take some time.
Approximate size to download 1.2 GB
[OK!]


In [35]:
nlp_pipeline = Pipeline(stages=[electra,ner_onto])

In [36]:
%time myNerModel = nlp_pipeline.fit(training_data)

CPU times: user 293 µs, sys: 941 µs, total: 1.23 ms
Wall time: 1.47 ms


In [37]:
%time results = myNerModel.transform(test).select("sentence","token","label","ner").collect()

CPU times: user 2.97 s, sys: 1.2 s, total: 4.17 s
Wall time: 22min 58s


In [38]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [39]:
print(count)
print(indices)

2
[3584, 3772]


In [40]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [41]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [42]:
print(accuracy_score(labels,ners))
print(f1_score(labels,ners))

0.9760530492844118
0.8774337268879482


In [43]:
print(classification_report(labels,ners, zero_division=1))

              precision    recall  f1-score   support

    CARDINAL       0.84      0.84      0.84       935
        DATE       0.85      0.87      0.86      1600
       EVENT       0.71      0.46      0.56        63
         FAC       0.63      0.53      0.58       135
         GPE       0.95      0.95      0.95      2240
    LANGUAGE       1.00      0.36      0.53        22
         LAW       0.62      0.50      0.56        40
         LOC       0.61      0.69      0.65       179
       MONEY       0.89      0.91      0.90       314
        NORP       0.87      0.95      0.91       841
     ORDINAL       0.81      0.91      0.86       195
         ORG       0.89      0.84      0.86      1791
     PERCENT       0.90      0.88      0.89       346
      PERSON       0.93      0.94      0.94      1988
     PRODUCT       0.77      0.63      0.70        76
    QUANTITY       0.77      0.81      0.79       105
        TIME       0.68      0.61      0.65       212
 WORK_OF_ART       0.64    

### Use pre-trained transformer model with BERT, this time casesensitive is True

In [44]:
ner_onto = NerDLModel.pretrained('onto_bert_base_cased', lang='en') \
        .setInputCols(["sentence", "token", "bert"])\
        .setOutputCol("ner")

onto_bert_base_cased download started this may take some time.
Approximate size to download 15.5 MB
[OK!]


In [62]:
# use bert embeddings
bert = BertEmbeddings.pretrained('bert_base_cased', 'en').setInputCols(["sentence",'token']).setOutputCol("bert").setCaseSensitive(True)#.setMaxSentenceLength(512)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In [63]:
nlp_pipeline = Pipeline(stages=[bert,ner_onto])

In [64]:
%time myNerModel = nlp_pipeline.fit(training_data)

CPU times: user 743 µs, sys: 453 µs, total: 1.2 ms
Wall time: 849 µs


In [65]:
%time results = myNerModel.transform(test).select("sentence","token","label","ner").collect()

CPU times: user 2.65 s, sys: 839 ms, total: 3.49 s
Wall time: 4min 48s


In [55]:
# to find exceptions where no. of labels does not match no. of ners detected
count = 0
indices = []
for i,row in enumerate(results):
    if len(row['label']) != len(row['ner']):
        count += 1
        indices.append(i)

In [56]:
print(count)
print(indices)

0
[]


In [57]:
exclusion_list = [results[t] for t in indices]
results = [results[i] for i in range(len(results)) if i not in indices]

In [58]:
tokens = []
labels = []
ners = []

for row in results:
    tokens.append([t['result'] for t in row['token']])
    labels.append([t['result'] for t in row['label']])
    ners.append([t['result'] for t in row['ner']])

In [59]:
print(accuracy_score(labels,ners))
print(f1_score(labels,ners))

0.979728667958724
0.8860354556360244


In [61]:
print(classification_report(labels,ners, zero_division=1,digits=6))

              precision    recall  f1-score   support

    CARDINAL   0.832827  0.879144  0.855359       935
        DATE   0.865262  0.845818  0.855429      1602
       EVENT   0.540984  0.523810  0.532258        63
         FAC   0.780488  0.711111  0.744186       135
         GPE   0.969266  0.943304  0.956109      2240
    LANGUAGE   0.909091  0.454545  0.606061        22
         LAW   0.785714  0.550000  0.647059        40
         LOC   0.796610  0.787709  0.792135       179
       MONEY   0.862500  0.878981  0.870662       314
        NORP   0.942217  0.950059  0.946122       841
     ORDINAL   0.815668  0.907692  0.859223       195
         ORG   0.865071  0.885794  0.875310      1795
     PERCENT   0.889535  0.876791  0.883117       349
      PERSON   0.934617  0.927565  0.931078      1988
     PRODUCT   0.710526  0.710526  0.710526        76
    QUANTITY   0.813084  0.828571  0.820755       105
        TIME   0.663551  0.669811  0.666667       212
 WORK_OF_ART   0.577778  0.

### Format the outputs

In [66]:
type(classification_report(labels,ners, zero_division=1,digits=6))

str

In [67]:
classification_report(labels,ners, zero_division=1,digits=6)

'              precision    recall  f1-score   support\n\n    CARDINAL   0.832827  0.879144  0.855359       935\n        DATE   0.865262  0.845818  0.855429      1602\n       EVENT   0.540984  0.523810  0.532258        63\n         FAC   0.780488  0.711111  0.744186       135\n         GPE   0.969266  0.943304  0.956109      2240\n    LANGUAGE   0.909091  0.454545  0.606061        22\n         LAW   0.785714  0.550000  0.647059        40\n         LOC   0.796610  0.787709  0.792135       179\n       MONEY   0.862500  0.878981  0.870662       314\n        NORP   0.942217  0.950059  0.946122       841\n     ORDINAL   0.815668  0.907692  0.859223       195\n         ORG   0.865071  0.885794  0.875310      1795\n     PERCENT   0.889535  0.876791  0.883117       349\n      PERSON   0.934617  0.927565  0.931078      1988\n     PRODUCT   0.710526  0.710526  0.710526        76\n    QUANTITY   0.813084  0.828571  0.820755       105\n        TIME   0.663551  0.669811  0.666667       212\n WORK_O

In [83]:
text = classification_report(labels,ners, zero_division=1,digits=6)

In [84]:
text = text.split("\n\n")

In [85]:
len(text)

3

In [110]:
columns = []
for t in text[0].strip().split("  "):
    if t == "":
        continue
    else:
        columns.append(t.strip())
columns = ["entity-type"] + columns

In [111]:
columns

['entity-type', 'precision', 'recall', 'f1-score', 'support']

In [118]:
new_rows = []
for sent in text[1].strip().split("\n"):
    temp = sent.split(" ")
    if "" in temp:
        while "" in temp:
            temp.remove("")
    new_rows.append(temp)

In [124]:
last_rows =[]
for sent in text[2].strip().split("\n"):
    temp = sent.split("  ")
    if "" in temp:
        while "" in temp:
            temp.remove("")
    last_rows.append([t.strip() for t in temp])

In [125]:
new_rows

[['CARDINAL', '0.832827', '0.879144', '0.855359', '935'],
 ['DATE', '0.865262', '0.845818', '0.855429', '1602'],
 ['EVENT', '0.540984', '0.523810', '0.532258', '63'],
 ['FAC', '0.780488', '0.711111', '0.744186', '135'],
 ['GPE', '0.969266', '0.943304', '0.956109', '2240'],
 ['LANGUAGE', '0.909091', '0.454545', '0.606061', '22'],
 ['LAW', '0.785714', '0.550000', '0.647059', '40'],
 ['LOC', '0.796610', '0.787709', '0.792135', '179'],
 ['MONEY', '0.862500', '0.878981', '0.870662', '314'],
 ['NORP', '0.942217', '0.950059', '0.946122', '841'],
 ['ORDINAL', '0.815668', '0.907692', '0.859223', '195'],
 ['ORG', '0.865071', '0.885794', '0.875310', '1795'],
 ['PERCENT', '0.889535', '0.876791', '0.883117', '349'],
 ['PERSON', '0.934617', '0.927565', '0.931078', '1988'],
 ['PRODUCT', '0.710526', '0.710526', '0.710526', '76'],
 ['QUANTITY', '0.813084', '0.828571', '0.820755', '105'],
 ['TIME', '0.663551', '0.669811', '0.666667', '212'],
 ['WORK_OF_ART', '0.577778', '0.626506', '0.601156', '166']]

In [126]:
last_rows

[['micro avg', '0.886311', '0.885760', '0.886035', '11257'],
 ['macro avg', '0.808599', '0.775430', '0.786290', '11257'],
 ['weighted avg', '0.887357', '0.885760', '0.886128', '11257']]

In [127]:
df_results = pd.DataFrame(new_rows + last_rows, columns=columns)

In [128]:
df_results

,entity-type,precision,recall,f1-score,support
0,CARDINAL,0.832827,0.879144,0.855359,935
1,DATE,0.865262,0.845818,0.855429,1602
2,EVENT,0.540984,0.523810,0.532258,63
3,FAC,0.780488,0.711111,0.744186,135
4,GPE,0.969266,0.943304,0.956109,2240
5,LANGUAGE,0.909091,0.454545,0.606061,22
6,LAW,0.785714,0.550000,0.647059,40
7,LOC,0.796610,0.787709,0.792135,179
8,MONEY,0.862500,0.878981,0.870662,314
9,NORP,0.942217,0.950059,0.946122,841


In [130]:
df_results.to_csv("/Users/ramybal/Desktop/temp.csv")